<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_hugectr_hps-hps-tensorflow-triton-deployment/nvidia_logo.png" style="width: 90px; float: right;">

# HPS TensorRT Plugin Demo for HugeCTR Trained Model

## Overview

This notebook demonstrates how to build and deploy the HPS-integrated TensorRT engine for the model trained with HugeCTR.

For more details about HPS, please refer to [HugeCTR Hierarchical Parameter Server (HPS)](https://nvidia-merlin.github.io/HugeCTR/master/hierarchical_parameter_server/index.html).

## Installation

### Use NGC

The HPS TensorRT plugin is preinstalled in the 23.01 and later [Merlin HugeCTR Container](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-hugectr): `nvcr.io/nvidia/merlin/merlin-hugectr:23.01`.

You can check the existence of the required libraries by running the following Python code after launching this container.

In [1]:
import ctypes
plugin_lib_name = "/usr/local/hps_trt/lib/libhps_plugin.so"
plugin_handle = ctypes.CDLL(plugin_lib_name, mode=ctypes.RTLD_GLOBAL)

## Data Generation

HugeCTR provides a tool to generate synthetic datasets. The [Data Generator](https://nvidia-merlin.github.io/HugeCTR/master/api/python_interface.html#data-generator-api) is capable of generating datasets of different file formats and different distributions. We will generate one-hot Parquet datasets with power-law distribution for this notebook:

In [2]:
import hugectr
from hugectr.tools import DataGeneratorParams, DataGenerator

data_generator_params = DataGeneratorParams(
  format = hugectr.DataReaderType_t.Parquet,
  label_dim = 1,
  dense_dim = 13,
  num_slot = 26,
  i64_input_key = True,
  nnz_array = [1 for _ in range(26)],
  source = "./data_parquet/file_list.txt",
  eval_source = "./data_parquet/file_list_test.txt",
  slot_size_array = [10000 for _ in range(26)],
  check_type = hugectr.Check_t.Non,
  dist_type = hugectr.Distribution_t.PowerLaw,
  power_law_type = hugectr.PowerLaw_t.Short,
  num_files = 16,
  eval_num_files = 4,
  num_samples_per_file = 40960)
data_generator = DataGenerator(data_generator_params)
data_generator.generate()

[HCTR][05:12:08.561][INFO][RK0][main]: Generate Parquet dataset
[HCTR][05:12:08.561][INFO][RK0][main]: train data folder: ./data_parquet, eval data folder: ./data_parquet, slot_size_array: 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, nnz array: 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, #files for train: 16, #files for eval: 4, #samples per file: 40960, Use power law distribution: 1, alpha of power law: 1.3
[HCTR][05:12:08.564][INFO][RK0][main]: ./data_parquet exist
[HCTR][05:12:08.568][INFO][RK0][main]: ./data_parquet/train/gen_0.parquet
[HCTR][05:12:10.204][INFO][RK0][main]: ./data_parquet/train/gen_1.parquet
[HCTR][05:12:10.455][INFO][RK0][main]: ./data_parquet/train/gen_2.parquet
[HCTR][05:12:10.709][INFO][RK0][main]: ./data_parquet/train/gen_3.parquet
[HCTR][05:12:10.957][INFO][RK0][main]: ./data_parquet/train/g

## Train with HugeCTR

We can train a DLRM model with HugeCTR Python APIs. The trained sparse and dense model files will be saved separately. The model graph will be dumped into a JSON file.

In [3]:
%%writefile train.py
import hugectr
from mpi4py import MPI


solver = hugectr.CreateSolver(
    model_name="dlrm",
    max_eval_batches=160,
    batchsize_eval=1024,
    batchsize=1024,
    lr=0.001,
    vvgpu=[[0]],
    repeat_dataset=True,
    use_mixed_precision=True,
    use_cuda_graph=True,
    scaler=1024,
    i64_input_key=True,
)
reader = hugectr.DataReaderParams(
    data_reader_type=hugectr.DataReaderType_t.Parquet,
    source=["./data_parquet/file_list.txt"],
    eval_source="./data_parquet/file_list_test.txt",
    slot_size_array=[10000 for _ in range(26)],
    check_type=hugectr.Check_t.Non,
)
optimizer = hugectr.CreateOptimizer(
    optimizer_type=hugectr.Optimizer_t.Adam,
    update_type=hugectr.Update_t.Global,
    beta1=0.9,
    beta2=0.999,
    epsilon=0.0001,
)

model = hugectr.Model(solver, reader, optimizer)
model.add(
    hugectr.Input(
        label_dim=1,
        label_name="label",
        dense_dim=13,
        dense_name="numerical_features",
        data_reader_sparse_param_array=[hugectr.DataReaderSparseParam("keys", 1, True, 26)],
    )
)
model.add(
    hugectr.SparseEmbedding(
        embedding_type=hugectr.Embedding_t.DistributedSlotSparseEmbeddingHash,
        workspace_size_per_gpu_in_mb=5000,
        embedding_vec_size=128,
        combiner="mean",
        sparse_embedding_name="sparse_embedding1",
        bottom_name="keys",
        optimizer=optimizer,
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.MLP,
        bottom_names=["numerical_features"],
        top_names=["mlp1"],
        num_outputs=[512, 256, 128],
        act_type=hugectr.Activation_t.Relu,
        use_bias=True,
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.Interaction,
        bottom_names=["mlp1", "sparse_embedding1"],
        top_names=["interaction1"],
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.MLP,
        bottom_names=["interaction1"],
        top_names=["mlp2"],
        num_outputs=[1024, 1024, 512, 256, 1],
        use_bias=True,
        activations=[
            hugectr.Activation_t.Relu,
            hugectr.Activation_t.Relu,
            hugectr.Activation_t.Relu,
            hugectr.Activation_t.Relu,
            hugectr.Activation_t.Non,
        ],
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.BinaryCrossEntropyLoss,
        bottom_names=["mlp2", "label"],
        top_names=["loss"],
    )
)
model.graph_to_json("dlrm_hugectr_graph.json")
model.compile()
model.summary()
model.fit(max_iter=1200, display=200, eval_interval=1000, snapshot=1000, snapshot_prefix="dlrm_hugectr")


Writing train.py


In [4]:
!python3 train.py

--------------------------------------------------------------------------
An error occurred while trying to map in the address of a function.
  Function Name: cuIpcOpenMemHandle_v2
  Error string:  /usr/lib/x86_64-linux-gnu/libcuda.so.1: undefined symbol: cuIpcOpenMemHandle_v2
CUDA-aware support is disabled.
--------------------------------------------------------------------------
HugeCTR Version: 4.1
====================================================Model Init=====================================================
[HCTR][05:12:24.539][INFO][RK0][main]: Initialize model: dlrm
[HCTR][05:12:24.539][INFO][RK0][main]: Global seed is 2950905596
[HCTR][05:12:24.542][INFO][RK0][main]: Device to NUMA mapping:
  GPU 0 ->  node 0
[HCTR][05:12:26.698][WARNING][RK0][main]: Peer-to-peer access cannot be fully enabled.
[HCTR][05:12:26.698][INFO][RK0][main]: Start all2all warmup
[HCTR][05:12:26.698][INFO][RK0][main]: End all2all warmup
[HCTR][05:12:26.699][INFO][RK0][main]: Using All-reduce algorit

## Build the HPS-integrated TensorRT engine
The sparse saved model `dlrm_hugectr0_sparse_1000.model` is already in the format that HPS requires. In order to use HPS in the inference stage, we need to create JSON configuration file for HPS.

Then we convert the dense saved model `dlrm_hugectr_dense_1000.model` to ONNX using `hugectr2onnx`, and employ the ONNX GraphSurgoen tool to replace the input embedding vectors with with the placeholder of HPS TensorRT plugin layer.

After that, we can build the TensorRT engine, which is comprised of the HPS TensorRT plugin layer and the dense network.

### Step1: Prepare JSON configuration file for HPS

Please note that the storage format in the `dlrm_hugectr0_sparse_1000.model/key` file is int64, while the HPS TensorRT plugin only supports int32 when loading the keys into memory. There is no overflow since the key value range is 0~260000.

In [5]:
%%writefile dlrm_hugectr.json
{
    "supportlonglong": false,
    "models": [{
        "model": "dlrm",
        "sparse_files": ["dlrm_hugectr0_sparse_1000.model"],
        "num_of_worker_buffer_in_pool": 3,
        "embedding_table_names":["sparse_embedding0"],
        "embedding_vecsize_per_table": [128],
        "maxnum_catfeature_query_per_table_per_sample": [26],
        "default_value_for_each_table": [1.0],
        "deployed_device_list": [0],
        "max_batch_size": 1024,
        "cache_refresh_percentage_per_iteration": 0.2,
        "hit_rate_threshold": 1.0,
        "gpucacheper": 1.0,
        "gpucache": true
        }
    ]
}

Writing dlrm_hugectr.json


### Step2: Convert to ONNX and do ONNX graph surgery

In [6]:
# hugectr2onnx
import hugectr2onnx
hugectr2onnx.converter.convert(onnx_model_path = "dlrm_hugectr_dense.onnx",
                            graph_config = "dlrm_hugectr_graph.json",
                            dense_model = "dlrm_hugectr_dense_1000.model",
                            convert_embedding = False)

[HUGECTR2ONNX][INFO]: Converting Data layer to ONNX
Skip sparse embedding layers in converted ONNX model
[HUGECTR2ONNX][INFO]: Converting DistributedSlotSparseEmbeddingHash layer to ONNX
[HUGECTR2ONNX][INFO]: Converting MLP layer to ONNX
[HUGECTR2ONNX][INFO]: Converting Interaction layer to ONNX
[HUGECTR2ONNX][INFO]: Converting MLP layer to ONNX
[HUGECTR2ONNX][INFO]: Converting Sigmoid layer to ONNX
[HUGECTR2ONNX][INFO]: The model is checked!
[HUGECTR2ONNX][INFO]: The model is saved at dlrm_hugectr_dense.onnx


In [7]:
# ONNX graph surgery to insert HPS the TensorRT plugin placeholder
import onnx_graphsurgeon as gs
from onnx import  shape_inference
import numpy as np
import onnx

graph = gs.import_onnx(onnx.load("dlrm_hugectr_dense.onnx"))
saved = []

for i in graph.inputs:
    if i.name == "sparse_embedding1":
        categorical_features = gs.Variable(name="categorical_features", dtype=np.int32, shape=("unknown_1", 26))
        node = gs.Node(op="HPS_TRT", attrs={"ps_config_file": "dlrm_hugectr.json\0", "model_name": "dlrm\0", "table_id": 0, "emb_vec_size": 128}, inputs=[categorical_features], outputs=[i])
        graph.nodes.append(node)
        saved.append(categorical_features)
    elif i.name == "numerical_features":
        i.shape = ("unknown_2", 13)
        saved.append(i)

graph.inputs = saved

graph.cleanup().toposort()
onnx.save(gs.export_onnx(graph), "dlrm_hugectr_with_hps.onnx")

### Step3: Build the TensorRT engine

In [8]:
# build the TensorRT engine based on dlrm_with_hps.onnx
import tensorrt as trt
import ctypes

plugin_lib_name = "/usr/local/hps_trt/lib/libhps_plugin.so"
handle = ctypes.CDLL(plugin_lib_name, mode=ctypes.RTLD_GLOBAL)

TRT_LOGGER = trt.Logger(trt.Logger.INFO)
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

def build_engine_from_onnx(onnx_model_path):
    with trt.Builder(TRT_LOGGER) as builder, builder.create_network(EXPLICIT_BATCH) as network, trt.OnnxParser(network, TRT_LOGGER) as parser, builder.create_builder_config() as builder_config:        
        model = open(onnx_model_path, 'rb')
        parser.parse(model.read())

        profile = builder.create_optimization_profile()        
        profile.set_shape("categorical_features", (1, 26), (1024, 26), (1024, 26))    
        profile.set_shape("numerical_features", (1, 13), (1024, 13), (1024, 13))
        builder_config.add_optimization_profile(profile)
        engine = builder.build_serialized_network(network, builder_config)
        return engine

serialized_engine = build_engine_from_onnx("dlrm_hugectr_with_hps.onnx")
with open("dlrm_hugectr_with_hps.trt", "wb") as fout:
    fout.write(serialized_engine)
print("Succesfully build the TensorRT engine")

[12/14/2022-05:13:31] [TRT] [I] [MemUsageChange] Init CUDA: CPU +262, GPU +0, now: CPU 1014, GPU 886 (MiB)
[12/14/2022-05:13:33] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +170, GPU +46, now: CPU 1239, GPU 932 (MiB)
[12/14/2022-05:13:33] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
[12/14/2022-05:13:33] [TRT] [W] onnx2trt_utils.cpp:377: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[12/14/2022-05:13:33] [TRT] [I] No importer registered for op: HPS_TRT. Attempting to import as plugin.
[12/14/2022-05:13:33] [TRT] [I] Searching for plugin: HPS_TRT, plugin_version: 1, plugin_namespace: 
=====================================================HPS Parse====================================================
[HCTR][05:13:33.812][INFO][

## Deploy HPS-integrated TensorRT engine on Triton

In order to deploy the TensorRT engine with the Triton TensorRT backend, we need to create the model repository and define the `config.pbtxt` first.

In [9]:
!mkdir -p model_repo/dlrm_hugectr_with_hps/1
!mv dlrm_hugectr_with_hps.trt model_repo/dlrm_hugectr_with_hps/1

In [10]:
%%writefile model_repo/dlrm_hugectr_with_hps/config.pbtxt

platform: "tensorrt_plan"
default_model_filename: "dlrm_hugectr_with_hps.trt"
backend: "tensorrt"
max_batch_size: 0
input [
  {
    name: "categorical_features"
    data_type: TYPE_INT32
    dims: [-1,26]
  },
  {
    name: "numerical_features"
    data_type: TYPE_FP32
    dims: [-1,13]
  }
]
output [
  {
      name: "label"
      data_type: TYPE_FP32
      dims: [-1,1]
  }
]
instance_group [
  {
    count: 1
    kind: KIND_GPU
    gpus:[0]

  }
]


Writing model_repo/dlrm_hugectr_with_hps/config.pbtxt


In [11]:
!tree model_repo/dlrm_hugectr_with_hps

model_repo/dlrm_hugectr_with_hps
├── 1
│   └── dlrm_hugectr_with_hps.trt
└── config.pbtxt

1 directory, 2 files


We can then launch the Triton inference server using the TensorRT backend. Please note that `LD_PRELOAD` is utilized to load the custom TensorRT plugin (i.e., HPS TensorRT plugin) into Triton.

Note: `Since Background processes not supported by Jupyter, please launch the Triton Server according to the following command independently in the background`.

> **LD_PRELOAD=/usr/local/hps_trt/lib/libhps_plugin.so tritonserver --model-repository=/hugectr/hps_trt/notebooks/model_repo/ --load-model=dlrm_hugectr_with_hps --model-control-mode=explicit**

If you successfully started tritonserver, you should see a log similar to following:


```bash
+----------+--------------------------------+--------------------------------+
| Backend  | Path                           | Config                         |
+----------+--------------------------------+--------------------------------+
| tensorrt | /opt/tritonserver/backends/ten | {"cmdline":{"auto-complete-con |
|          | sorrt/libtriton_tensorrt.so    | fig":"true","min-compute-capab |
|          |                                | ility":"6.000000","backend-dir |
|          |                                | ectory":"/opt/tritonserver/bac |
|          |                                | kends","default-max-batch-size |
|          |                                | ":"4"}}                        |
|          |                                |                                |
+----------+--------------------------------+--------------------------------+

+-----------------------+---------+--------+
| Model                 | Version | Status |
+-----------------------+---------+--------+
| dlrm_hugectr_with_hps | 1       | READY  |
+-----------------------+---------+--------+
```

We can then send the requests to the Triton inference server using the HTTP client.

In [12]:
import os
import shutil
import numpy as np
import tritonclient.http as httpclient
from tritonclient.utils import *

BATCH_SIZE = 1024

categorical_feature = np.random.randint(0,260000,size=(BATCH_SIZE,26)).astype(np.int32)
numerical_feature = np.random.random((BATCH_SIZE, 13)).astype(np.float32)

inputs = [
    httpclient.InferInput("categorical_features", 
                          categorical_feature.shape,
                          np_to_triton_dtype(np.int32)),
    httpclient.InferInput("numerical_features", 
                          numerical_feature.shape,
                          np_to_triton_dtype(np.float32)),                          
]
inputs[0].set_data_from_numpy(categorical_feature)
inputs[1].set_data_from_numpy(numerical_feature)


outputs = [
    httpclient.InferRequestedOutput("label")
]

model_name = "dlrm_hugectr_with_hps"

with httpclient.InferenceServerClient("localhost:8000") as client:
    response = client.infer(model_name,
                            inputs,
                            outputs=outputs)
    result = response.get_response()
    
    print("Prediction result is \n{}".format(response.as_numpy("label")))
    print("Response details:\n{}".format(result))


Prediction result is 
[[1.        ]
 [0.49642828]
 [0.52846366]
 ...
 [0.99999994]
 [0.9999992 ]
 [0.9999905 ]]
Response details:
{'model_name': 'dlrm_hugectr_with_hps', 'model_version': '1', 'outputs': [{'name': 'label', 'datatype': 'FP32', 'shape': [1024, 1], 'parameters': {'binary_data_size': 4096}}]}
